# Book SCR - Processamento de Safra
Notebook utilizado para processamento dos dados de SCR para criação do book, organizando por safra (ano mês) os dados.
Esse processamento é construído a partir de duas tabelas de entrada, **scr_clientes** e **scr_vencimentos**, filtradas pela safra de interesse. Essas tabelas têm uma defasagem de 2 meses (tempo de disponibilização do BACEN dos dados), logo, o processamento feito em novembro de 2021, gerará os dados de SCR de setembro de 2021. Para facilitar a construção do código, são criadas tabelas temporárias auxiliares, **SCR_VENCIMENTOS_AUX**, **SCR_CLIENTES_AUX** e **BOOK_SCR_AUX**.

A tabela de book a ser criada nesse processamento, **BOOK_SCR_AAAAMM** (sendo AAAAMM o ano e mês de interesse), contém as informações de vencimentos divididas por 18 modalidades de produtos, assim como a divisão em diferentes faixas de tempo de vencimento e conceitos criados pelo BACEN (Carteira Ativa, de crédito, Coobrigação, etc.).

In [0]:
from datetime import date
from dateutil.relativedelta import relativedelta

data_atual = date.today()
data_2_meses_anterior = data_atual - relativedelta(months=2)
safra_consumo_scr = data_2_meses_anterior.strftime("%Y%m")

TABELA_SCR_CLIENTES = 'inf_trustedzone.scr_clientes'
TABELA_SCR_VENCIMENTOS = 'inf_trustedzone.scr_vencimentos'

TABELA_SCR_CLIENTES_AUX = 'default.scr_clientes_aux'
TABELA_SCR_VENCIMENTOS_AUX = 'default.scr_vencimentos_aux'

TABELA_SCR_AUX = 'default.BOOK_SCR_AUX'
TABELA_SCR_FINAL = 'default.BOOK_SCR_'+safra_consumo_scr

In [0]:
print('Nome das colunas trabalhadas neste notebook (em função da data atual de processamento):')
print(TABELA_SCR_ORIGEM_CLIENTES)
print(TABELA_SCR_ORITEM_VENCIMENTOS)
print() 

print('Tabelas Auxiliares:')
print(TABELA_SCR_CLIENTES_AUX)
print(TABELA_SCR_VENCIMENTOS_AUX)
print(TABELA_SCR_AUX)
print()

print(TABELA_SCR_FINAL)

Nome das colunas trabalhadas neste notebook (em função da data atual de processamento):
inf_trustedzone.scr_clientes
inf_trustedzone.scr_vencimentos

Tabelas Auxiliares:
default.scr_clientes_aux
default.scr_vencimentos_aux
default.BOOK_SCR_AUX

default.BOOK_SCR_202109

In [0]:
query_scr_vencimentos = '''CREATE OR REPLACE TEMP VIEW''' + TABELA_SCR_VENCIMENTOS_AUX + ''' AS
SELECT * 
FROM ''' + TABELA_SCR_CLIENTES + '''
WHERE dtbase==''' + safra_consumo_scr

query_scr_clientes = '''CREATE OR REPLACE TEMP VIEW''' + TABELA_SCR_CLIENTES_AUX + ''' AS
SELECT * 
FROM ''' + TABELA_SCR_VENCIMENTOS + '''
WHERE dtbase==''' + safra_consumo_scr

spark.sql(query_scr_vencimentos)
spark.sql(query_scr_clientes)

Out[15]: DataFrame[]

In [0]:
query_join_clientes_vencimentos = '''
create or replace temp view BOOK_SCR_AUX as
select vencimentos.dtbase as dt_base
     ,cast(date_Format(add_months(to_date(substr(cast(vencimentos.dtbase as string),1,4) ||'-'|| substr(cast(vencimentos.dtbase as string),5,2) || '-01'),2),'yyyyMM') as int) as dt_modelagem
     ,from_unixtime(unix_timestamp(), 'yyyyMMdd') as dt_processamento
     ,case when tipopessoa = 1 then 'F' else 'J' end as tipopessoa
     ,vencimentos.cpf                     
     ,cod_modalidade
     ,case when cod_modalidade in (1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1999) then 'Limite'
           when cod_modalidade in (101) then 'Adp'

           when cod_modalidade in (201, 213, 214) then 'ChqCcg'
           when cod_modalidade in (204, 218, 210, 406, 1304) then 'Car'
           when cod_modalidade in (401, 1206) then 'Auto'
           when cod_modalidade in (211, 402, 901, 902, 903, 990, 1202) then 'Imob'
           when cod_modalidade in (202) then 'Consig'
           when cod_modalidade in (203) then 'EP'
           when cod_modalidade in (299) then 'SemGarOut'

           when cod_modalidade in (206, 215, 216, 217) then 'CGiro'
           when cod_modalidade in (301) then 'DesDup'
           when cod_modalidade in (302) then 'DesChq'
           when cod_modalidade in (212, 403) then 'Micro'
           when cod_modalidade in (211, 401, 402, 801, 802, 901, 902, 903, 990, 1201, 1202, 1206) then 'ComGar'
           when cod_modalidade in (202, 203, 299) then 'SemGar'
           when cod_modalidade in (202, 203, 211, 299, 401, 402, 403, 499, 1201, 1206, 1301, 1401) then 'EmpFinPri'
           when cod_modalidade in (202, 203, 207, 211, 250, 290, 299, 401, 402, 404, 405, 440, 450, 490, 499, 501, 502, 503, 504, 590, 599, 601, 702, 801, 802, 803, 890, 901, 902, 903, 990, 1001, 1101, 1201, 1202, 1205, 1206, 1301, 1302, 1303, 1350, 1399, 1401) then 'EmpFin'

           else 'Outro' end as Cl_Produto

     --Pagamentos a vencer até 30+ dias (o retorno 3046 inclui o saldo vencido de 1 a 14 dias - 205) - 110
     ,sum(case when vencimentos.cod_vencimento = 110 then vencimentos.vl_vencimento else 0 end) as vl_avencer_30d
     ,sum(case when vencimentos.cod_vencimento in (205, 110) then vencimentos.vl_vencimento else 0 end) as vl_avencer30d_venc14d -- DÚVIDA: por que a criação dessa coluna? Representaria o total de créditos a vencer e vencidos, até os respectivos dias (30d pra a vencer, e 14d para vencido)?

     ,sum(case when vencimentos.cod_vencimento = 120 then vencimentos.vl_vencimento else 0 end) as vl_avencer_31a60d
     ,sum(case when vencimentos.cod_vencimento = 130 then vencimentos.vl_vencimento else 0 end) as vl_avencer_61a90d
     ,sum(case when vencimentos.cod_vencimento = 140 then vencimentos.vl_vencimento else 0 end) as vl_avencer_91a180d
     ,sum(case when vencimentos.cod_vencimento = 150 then vencimentos.vl_vencimento else 0 end) as vl_avencer_181a360d
     
     ,sum(case when vencimentos.cod_vencimento between 160 and 190 then vencimentos.vl_vencimento else 0 end) as vl_avencer_acima360d -- MODIFICAR (ABRIR OS VENCIMENTOS)
     ,sum(case when vencimentos.cod_vencimento = 160 then vencimentos.vl_vencimento else 0 end) as vl_avencer_361a720d
     ,sum(case when vencimentos.cod_vencimento = 165 then vencimentos.vl_vencimento else 0 end) as vl_avencer_721a1080d
     ,sum(case when vencimentos.cod_vencimento = 170 then vencimentos.vl_vencimento else 0 end) as vl_avencer_1081a1440d
     ,sum(case when vencimentos.cod_vencimento = 175 then vencimentos.vl_vencimento else 0 end) as vl_avencer_1441a1800d
     ,sum(case when vencimentos.cod_vencimento = 180 then vencimentos.vl_vencimento else 0 end) as vl_avencer_1801a5400d
     ,sum(case when vencimentos.cod_vencimento = 190 then vencimentos.vl_vencimento else 0 end) as vl_avencer_acima_5400d

     --Total a vencer (o retorno 3046 inclui o saldo vencido de 1 a 14 dias - 205) - 110
     ,sum(case when ((vencimentos.cod_vencimento between 110 and 199) or (vencimentos.cod_vencimento in (60,80) )) then vencimentos.vl_vencimento else 0 end) as vl_avencer
     ,sum(case when ((vencimentos.cod_vencimento between 110 and 199) or (vencimentos.cod_vencimento in (60,80,205) )) then vencimentos.vl_vencimento else 0 end) as vl_avencer_venc14d

     --Saldo em atraso 1+ dias de atraso - retorno não tem essa informação
     ,sum(case when vencimentos.cod_vencimento between 205 and 290 then vencimentos.vl_vencimento else 0 end) as vl_venc_1d

     -- Saldo em atraso 15+..180+ dias de atraso
     ,sum(case when vencimentos.cod_vencimento between 210 and 290 then vencimentos.vl_vencimento else 0 end) as vl_venc_15d  -- VENCIDO
     ,sum(case when vencimentos.cod_vencimento between 220 and 290 then vencimentos.vl_vencimento else 0 end) as vl_venc_30d
     ,sum(case when vencimentos.cod_vencimento between 230 and 290 then vencimentos.vl_vencimento else 0 end) as vl_venc_60d
     ,sum(case when vencimentos.cod_vencimento between 240 and 290 then vencimentos.vl_vencimento else 0 end) as vl_venc_90d
     ,sum(case when vencimentos.cod_vencimento between 245 and 290 then vencimentos.vl_vencimento else 0 end) as vl_venc_120d                                        
     ,sum(case when vencimentos.cod_vencimento between 255 and 290 then vencimentos.vl_vencimento else 0 end) as vl_venc_180d --DÚVIDA: Incluir os outros códigos (255 a 290)?

     --Valor em Prejuízo baixado nos últimos 48 meses
     ,sum(case when vencimentos.cod_vencimento between 310 and 320 then vencimentos.vl_vencimento else 0 end) as vl_prejuizo_ult48m

     --Valor em Prejuízo - retorno não tem essa informação 330
     ,sum(case when vencimentos.cod_vencimento between 310 and 330 then vencimentos.vl_vencimento else 0 end) as vl_prejuizo

     --Saldo devedor Mensal
     ,sum(case when ((vencimentos.cod_vencimento = 110) or (vencimentos.cod_vencimento between 205 and 290)) then vencimentos.vl_vencimento else 0 end) as vl_sld_devedor_mensal

     --Saldo devedor Total
     ,sum(case when ((vencimentos.cod_vencimento between 60 and 199) or (vencimentos.cod_vencimento between 205 and 290)) then vencimentos.vl_vencimento else 0 end) as vl_sld_devedor_total

     --Total de limite disponível
     ,sum(case when vencimentos.cod_vencimento in (20, 40) then vencimentos.vl_vencimento else 0 end) as vl_limite_disponivel
     
     --CALCULOS BACEN
     
     --Carteira Ativa
     ,sum(case when (vencimentos.cod_vencimento between 110 and 290) and (vencimentos.cod_modalidade between 101 and 1399) then vencimentos.vl_vencimento else 0 end) as Bacen_vl_carteira_ativa
     
     --Carteira Credito
     ,sum(case when (vencimentos.cod_vencimento between 110 and 330) and (vencimentos.cod_modalidade between 101 and 1499) then vencimentos.vl_vencimento else 0 end) as Bacen_vl_carteira_credito
     
     --Relacoes Interfinanceiras
     ,sum(case when (vencimentos.cod_vencimento between 60 and 290) and (vencimentos.cod_modalidade between 1401 and 1499) then vencimentos.vl_vencimento else 0 end) as Bacen_vl_relac_interfin
     
     --Garantias Prestadas
     ,sum(case when (vencimentos.cod_vencimento between 110 and 290) and (vencimentos.cod_modalidade between 1501 and 1599) and (vencimentos.cod_modalidade not in (1511,1512,1513) ) then vencimentos.vl_vencimento else 0 end) as Bacen_vl_gar_prestadas
     
     --Coobigacao Assumida
     ,max(clientes.coobass) as Bacen_vl_coobigacao_assumida
     
     --Coobrigação
     ,sum(case when (vencimentos.cod_vencimento between 110 and 290) and (vencimentos.cod_modalidade between 1501 and 1599) and (vencimentos.cod_modalidade not in (1511,1512,1513) ) then vencimentos.vl_vencimento else 0 end) + max(clientes.coobass) as Bacen_vl_coobricacao
     
     --Responsabilidade Total (Carteira Credito + Relacoes Interfinanceiras + Coobrigação)
     ,(sum(case when (vencimentos.cod_vencimento between 110 and 330) and (vencimentos.cod_modalidade between 101 and 1499) then vencimentos.vl_vencimento else 0 end) + 
       sum(case when (vencimentos.cod_vencimento between 60 and 290) and (vencimentos.cod_modalidade between 1401 and 1499) then vencimentos.vl_vencimento else 0 end) + 
       sum(case when (vencimentos.cod_vencimento between 110 and 290) and (vencimentos.cod_modalidade between 1501 and 1599) and (vencimentos.cod_modalidade not in (1511,1512,1513) ) then vencimentos.vl_vencimento else 0 end) + max(clientes.coobass)) as Bacen_vl_resp_total
      
     --Limite de Credito
     ,sum(case when (vencimentos.cod_vencimento between 20 and 40) and (vencimentos.cod_modalidade between 1901 and 1999) then vencimentos.vl_vencimento else 0 end) as Bacen_vl_limite_credito
     
     --Creditos a Liberar
     ,sum(case when (vencimentos.cod_vencimento between 60 and 80) and (vencimentos.cod_modalidade between 101 and 1399) then vencimentos.vl_vencimento else 0 end) as Bacen_vl_creditos_a_liberar
     
     --Risco Indireto
     ,max(clientes.riscoindvendor) as Bacen_risco_indireto
     
     --Risco Total (Resp Total + Lim. Credito + Cred. a Liberar + Risco Indireto)
     ,((sum(case when (vencimentos.cod_vencimento between 110 and 330) and (vencimentos.cod_modalidade between 101 and 1499) then vencimentos.vl_vencimento else 0 end) + 
       sum(case when (vencimentos.cod_vencimento between 60 and 290) and (vencimentos.cod_modalidade between 1401 and 1499) then vencimentos.vl_vencimento else 0 end) + 
       sum(case when (vencimentos.cod_vencimento between 110 and 290) and (vencimentos.cod_modalidade between 1501 and 1599) and (vencimentos.cod_modalidade not in (1511,1512,1513) ) then vencimentos.vl_vencimento else 0 end) + max(clientes.coobass)) + 
       sum(case when (vencimentos.cod_vencimento between 20 and 40) and (vencimentos.cod_modalidade between 1901 and 1999) then vencimentos.vl_vencimento else 0 end) + 
       sum(case when (vencimentos.cod_vencimento between 60 and 80) and (vencimentos.cod_modalidade between 101 and 1399) then vencimentos.vl_vencimento else 0 end) + 
       max(clientes.riscoindvendor)) as Bacen_risco_total
     
     --Quantidade de Instituições Financeiras
     ,max(clientes.qtdif) as qtdIF
     
     -- Tempo relac. Instituições Financeiras
     ,round(months_between(to_date(substr(cast(max(vencimentos.dtbase) as string),1,4) ||'-'|| substr(cast(max(vencimentos.dtbase) as string),5,2) || '-01'),max(to_date(clientes.inirelactcli))),2) as meses_relac_if 
   
from ''' + TABELA_SCR_VENCIMENTOS_AUX + ''' vencimentos
left join ''' + TABELA_SCR_CLIENTES_AUX + ''' clientes
  on vencimentos.dtbase=cast(replace(clientes.dtbase,'-','') as int)
 and vencimentos.cpf=clientes.cpf
 --and vencimentos.tipopessoa=(case when clientes.tipopessoa = 'PF' then 1 else 2 end) -- ATENÇÃO AQUI
 where from_unixtime(unix_timestamp(CAST(vencimentos.dtbase as STRING), 'yyyyMM'), 'MM')==MONTH((add_months(current_date(), -2)))
group by 1,2,3,4,5,6
'''

spark.sql(query_join_clientes_vencimentos)

In [0]:
query_book_scr = '''
create or replace table ''' + TABELA_SCR_FINAL + ''' as 
select  
       dt_base
       ,dt_modelagem
       ,dt_processamento
       ,tipopessoa
       ,cpf
       
       ,sum(vl_avencer_30d)        as vl_avencer_30d        
       ,sum(vl_avencer_31a60d)     as vl_avencer_31a60d     
       ,sum(vl_avencer_61a90d)     as vl_avencer_61a90d     
       ,sum(vl_avencer_91a180d)    as vl_avencer_91a180d    
       ,sum(vl_avencer_181a360d)   as vl_avencer_181a360d
       ,sum(vl_avencer_acima360d)  as vl_avencer_acima360d
       ,sum(vl_avencer_361a720d)   as vl_avencer_361a720d
       ,sum(vl_avencer_721a1080d)   as vl_avencer_721a1080d
       ,sum(vl_avencer_1081a1440d)   as vl_avencer_1081a1440d
       ,sum(vl_avencer_1441a1800d)   as vl_avencer_1441a1800d
       ,sum(vl_avencer_1801a5400d)   as vl_avencer_1801a5400d
       ,sum(vl_avencer_acima_5400d)   as vl_avencer_acima_5400d

       ,sum(vl_avencer)            as vl_avencer            
       ,sum(vl_venc_1d)            as vl_venc_1d            
       ,sum(vl_prejuizo)           as vl_prejuizo           
     
       ,sum(vl_avencer30d_venc14d) as vl_avencer30d_venc14d 
       ,sum(vl_avencer_venc14d)    as vl_avencer_venc14d    
       
       ,sum(vl_venc_15d)           as vl_venc_15d           
       ,sum(vl_venc_30d)           as vl_venc_30d           
       ,sum(vl_venc_60d)           as vl_venc_60d           
       ,sum(vl_venc_90d)           as vl_venc_90d           
       ,sum(vl_venc_120d)          as vl_venc_120d          
       ,sum(vl_venc_180d)          as vl_venc_180d          
     
       ,sum(vl_prejuizo_ult48m)    as vl_prejuizo_ult48m    
      
       ,sum(vl_sld_devedor_mensal) as vl_sld_devedor_mensal 
       ,sum(vl_sld_devedor_total)  as vl_sld_devedor_total
       
       ,sum(vl_venc_15d) / sum(vl_sld_devedor_total)                                    as Pc_Sd_venc_Por_Total 
       ,sum(vl_prejuizo_ult48m) / (sum(vl_prejuizo_ult48m) + sum(vl_sld_devedor_total)) as Pc_Sd_Prej_Por_Total

       ,count(distinct Cl_Produto) as Qt_Produtos
       
       --MOD 101 (Adiantamentos a Depositantes (ADP))**************************************************************************************************************************************
       ,sum(case when cod_modalidade = 101 then vl_avencer30d_venc14d else 0 end) as Adp_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade = 101 then vl_avencer_30d        else 0 end) as Adp_vl_avencer_30d
       ,sum(case when cod_modalidade = 101 then vl_avencer_31a60d     else 0 end) as Adp_vl_avencer_31a60d 
       ,sum(case when cod_modalidade = 101 then vl_avencer_61a90d     else 0 end) as Adp_vl_avencer_61a90d 
       ,sum(case when cod_modalidade = 101 then vl_avencer_91a180d    else 0 end) as Adp_vl_avencer_91a180d 
       ,sum(case when cod_modalidade = 101 then vl_avencer_181a360d   else 0 end) as Adp_vl_avencer_181a360d
       ,sum(case when cod_modalidade = 101 then vl_avencer_acima360d  else 0 end) as Adp_vl_avencer_acima360d -- MODIFICAR (ABRIR OS VENCIMETNOS)
       ,sum(case when cod_modalidade = 101 then vl_avencer_361a720d   else 0 end) as Adp_vl_avencer_361a720d
       ,sum(case when cod_modalidade = 101 then vl_avencer_721a1080d   else 0 end) as Adp_vl_avencer_721a1080d
       ,sum(case when cod_modalidade = 101 then vl_avencer_1081a1440d   else 0 end) as Adp_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade = 101 then vl_avencer_1441a1800d   else 0 end) as Adp_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade = 101 then vl_avencer_1801a5400d   else 0 end) as Adp_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade = 101 then vl_avencer_acima_5400d   else 0 end) as Adp_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade = 101 then vl_avencer            else 0 end) as Adp_vl_avencer 
       ,sum(case when cod_modalidade = 101 then vl_venc_15d           else 0 end) as Adp_vl_venc_15d 
       ,sum(case when cod_modalidade = 101 then vl_venc_30d           else 0 end) as Adp_vl_venc_30d 
       ,sum(case when cod_modalidade = 101 then vl_venc_60d           else 0 end) as Adp_vl_venc_60d 
       ,sum(case when cod_modalidade = 101 then vl_venc_90d           else 0 end) as Adp_vl_venc_90d 
       ,sum(case when cod_modalidade = 101 then vl_sld_devedor_mensal else 0 end) as Adp_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade = 101 then vl_sld_devedor_total  else 0 end) as Adp_vl_sld_devedor_total
       
       --MOD 201 213 214 (Cheque ou Conta Garantida)**************************************************************************************************************************************
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer30d_venc14d else 0 end) as chqCcg_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_30d        else 0 end) as chqCcg_vl_avencer_30d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_31a60d     else 0 end) as chqCcg_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_61a90d     else 0 end) as chqCcg_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_91a180d    else 0 end) as chqCcg_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_181a360d   else 0 end) as chqCcg_vl_avencer_181a360d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_acima360d  else 0 end) as chqCcg_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_361a720d   else 0 end) as chqCcg_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_721a1080d   else 0 end) as chqCcg_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_1081a1440d   else 0 end) as chqCcg_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_1441a1800d   else 0 end) as chqCcg_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_1801a5400d   else 0 end) as chqCcg_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer_acima_5400d   else 0 end) as chqCcg_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (201,213,214) then vl_avencer            else 0 end) as chqCcg_vl_avencer 
       ,sum(case when cod_modalidade in (201,213,214) then vl_venc_15d           else 0 end) as chqCcg_vl_venc_15d 
       ,sum(case when cod_modalidade in (201,213,214) then vl_venc_30d           else 0 end) as chqCcg_vl_venc_30d 
       ,sum(case when cod_modalidade in (201,213,214) then vl_venc_60d           else 0 end) as chqCcg_vl_venc_60d 
       ,sum(case when cod_modalidade in (201,213,214) then vl_venc_90d           else 0 end) as chqCcg_vl_venc_90d 
       ,sum(case when cod_modalidade in (201,213,214) then vl_sld_devedor_mensal else 0 end) as chqCcg_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (201,213,214) then vl_sld_devedor_total  else 0 end) as chqCcg_vl_sld_devedor_total
       
       --MOD 101 201 213 214 (Adp ou Cheque ou Conta Garantida)**************************************************************************************************************************************
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer30d_venc14d else 0 end) as AdpChqCcg_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_30d        else 0 end) as AdpChqCcg_vl_avencer_30d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_31a60d     else 0 end) as AdpChqCcg_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_61a90d     else 0 end) as AdpChqCcg_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_91a180d    else 0 end) as AdpChqCcg_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_181a360d   else 0 end) as AdpChqCcg_vl_avencer_181a360d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_acima360d  else 0 end) as AdpChqCcg_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_361a720d   else 0 end) as AdpChqCcg_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_721a1080d   else 0 end) as AdpChqCcg_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_1081a1440d   else 0 end) as AdpChqCcg_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_1441a1800d   else 0 end) as AdpChqCcg_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_1801a5400d   else 0 end) as AdpChqCcg_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer_acima_5400d   else 0 end) as AdpChqCcg_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_avencer            else 0 end) as AdpChqCcg_vl_avencer 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_venc_15d           else 0 end) as AdpChqCcg_vl_venc_15d 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_venc_30d           else 0 end) as AdpChqCcg_vl_venc_30d 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_venc_60d           else 0 end) as AdpChqCcg_vl_venc_60d 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_venc_90d           else 0 end) as AdpChqCcg_vl_venc_90d 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_sld_devedor_mensal else 0 end) as AdpChqCcg_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_sld_devedor_total  else 0 end) as AdpChqCcg_vl_sld_devedor_total
       
       --MOD 204 218 (Cartão (rotativo))**************************************************************************************************************************************
       ,sum(case when cod_modalidade in (204,218) then vl_avencer30d_venc14d else 0 end) as CarRot_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_30d        else 0 end) as CarRot_vl_avencer_30d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_31a60d     else 0 end) as CarRot_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_61a90d     else 0 end) as CarRot_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_91a180d    else 0 end) as CarRot_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_181a360d   else 0 end) as CarRot_vl_avencer_181a360d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_acima360d  else 0 end) as CarRot_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_361a720d   else 0 end) as CarRot_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_721a1080d   else 0 end) as CarRot_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_1081a1440d   else 0 end) as CarRot_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_1441a1800d   else 0 end) as CarRot_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_1801a5400d   else 0 end) as CarRot_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (204,218) then vl_avencer_acima_5400d   else 0 end) as CarRot_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (204,218) then vl_avencer            else 0 end) as CarRot_vl_avencer 
       ,sum(case when cod_modalidade in (204,218) then vl_venc_15d           else 0 end) as CarRot_vl_venc_15d 
       ,sum(case when cod_modalidade in (204,218) then vl_venc_30d           else 0 end) as CarRot_vl_venc_30d 
       ,sum(case when cod_modalidade in (204,218) then vl_venc_60d           else 0 end) as CarRot_vl_venc_60d 
       ,sum(case when cod_modalidade in (204,218) then vl_venc_90d           else 0 end) as CarRot_vl_venc_90d 
       ,sum(case when cod_modalidade in (204,218) then vl_sld_devedor_mensal else 0 end) as CarRot_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (204,218) then vl_sld_devedor_total  else 0 end) as CarRot_vl_sld_devedor_total
       
       --MOD 210 406 (Cartão (compra, fatura parcelada ou saque financiado pela instituição))**************************************************************************************************************************************
       ,sum(case when cod_modalidade in (210,406) then vl_avencer30d_venc14d else 0 end) as CarSaqLoj_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_30d        else 0 end) as CarSaqLoj_vl_avencer_30d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_31a60d     else 0 end) as CarSaqLoj_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_61a90d     else 0 end) as CarSaqLoj_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_91a180d    else 0 end) as CarSaqLoj_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_181a360d   else 0 end) as CarSaqLoj_vl_avencer_181a360d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_acima360d  else 0 end) as CarSaqLoj_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_361a720d   else 0 end) as CarSaqLoj_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_721a1080d   else 0 end) as CarSaqLoj_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_1081a1440d   else 0 end) as CarSaqLoj_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_1441a1800d   else 0 end) as CarSaqLoj_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_1801a5400d   else 0 end) as CarSaqLoj_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (210,406) then vl_avencer_acima_5400d   else 0 end) as CarSaqLoj_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (210,406) then vl_avencer            else 0 end) as CarSaqLoj_vl_avencer 
       ,sum(case when cod_modalidade in (210,406) then vl_venc_15d           else 0 end) as CarSaqLoj_vl_venc_15d 
       ,sum(case when cod_modalidade in (210,406) then vl_venc_30d           else 0 end) as CarSaqLoj_vl_venc_30d 
       ,sum(case when cod_modalidade in (210,406) then vl_venc_60d           else 0 end) as CarSaqLoj_vl_venc_60d 
       ,sum(case when cod_modalidade in (210,406) then vl_venc_90d           else 0 end) as CarSaqLoj_vl_venc_90d 
       ,sum(case when cod_modalidade in (210,406) then vl_sld_devedor_mensal else 0 end) as CarSaqLoj_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (210,406) then vl_sld_devedor_total  else 0 end) as CarSaqLoj_vl_sld_devedor_total
       
       --MOD 1304 (Cartão (compra à vista e parcelado lojista)))**************************************************************************************************************************************
       ,sum(case when cod_modalidade = 1304 then vl_avencer30d_venc14d else 0 end) as CarVisPar_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_30d        else 0 end) as CarVisPar_vl_avencer_30d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_31a60d     else 0 end) as CarVisPar_vl_avencer_31a60d 
       ,sum(case when cod_modalidade = 1304 then vl_avencer_61a90d     else 0 end) as CarVisPar_vl_avencer_61a90d 
       ,sum(case when cod_modalidade = 1304 then vl_avencer_91a180d    else 0 end) as CarVisPar_vl_avencer_91a180d 
       ,sum(case when cod_modalidade = 1304 then vl_avencer_181a360d   else 0 end) as CarVisPar_vl_avencer_181a360d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_acima360d  else 0 end) as CarVisPar_vl_avencer_acima360d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_361a720d   else 0 end) as CarVisPar_vl_avencer_361a720d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_721a1080d   else 0 end) as CarVisPar_vl_avencer_721a1080d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_1081a1440d   else 0 end) as CarVisPar_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_1441a1800d   else 0 end) as CarVisPar_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_1801a5400d   else 0 end) as CarVisPar_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade = 1304 then vl_avencer_acima_5400d   else 0 end) as CarVisPar_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade = 1304 then vl_avencer            else 0 end) as CarVisPar_vl_avencer 
       ,sum(case when cod_modalidade = 1304 then vl_venc_15d           else 0 end) as CarVisPar_vl_venc_15d 
       ,sum(case when cod_modalidade = 1304 then vl_venc_30d           else 0 end) as CarVisPar_vl_venc_30d 
       ,sum(case when cod_modalidade = 1304 then vl_venc_60d           else 0 end) as CarVisPar_vl_venc_60d 
       ,sum(case when cod_modalidade = 1304 then vl_venc_90d           else 0 end) as CarVisPar_vl_venc_90d 
       ,sum(case when cod_modalidade = 1304 then vl_sld_devedor_mensal else 0 end) as CarVisPar_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade = 1304 then vl_sld_devedor_total  else 0 end) as CarVisPar_vl_sld_devedor_total
       
       --MOD 204 218 210 406 1304 (Cartão (rotativo ou saque ou compra parcelada lojista ou saque ou compra parcelada lojista)))******************************************************************************
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer30d_venc14d else 0 end) as Car_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_30d        else 0 end) as Car_vl_avencer_30d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_31a60d     else 0 end) as Car_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_61a90d     else 0 end) as Car_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_91a180d    else 0 end) as Car_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_181a360d   else 0 end) as Car_vl_avencer_181a360d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_acima360d  else 0 end) as Car_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_361a720d   else 0 end) as Car_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_721a1080d   else 0 end) as Car_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_1081a1440d   else 0 end) as Car_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_1441a1800d   else 0 end) as Car_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_1801a5400d   else 0 end) as Car_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer_acima_5400d   else 0 end) as Car_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_avencer            else 0 end) as Car_vl_avencer 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_venc_15d           else 0 end) as Car_vl_venc_15d 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_venc_30d           else 0 end) as Car_vl_venc_30d 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_venc_60d           else 0 end) as Car_vl_venc_60d 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_venc_90d           else 0 end) as Car_vl_venc_90d 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_sld_devedor_mensal else 0 end) as Car_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_sld_devedor_total  else 0 end) as Car_vl_sld_devedor_total
       
       --MOD 101 201 213 214 204 218 210 406 1304 (Rotativos (Adp ou Cheque ou Conta Garantida ou Cartão)))******************************************************************************
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer30d_venc14d else 0 end) as Rotv_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_30d        else 0 end) as Rotv_vl_avencer_30d
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_31a60d     else 0 end) as Rotv_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_61a90d     else 0 end) as Rotv_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_91a180d    else 0 end) as Rotv_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_181a360d   else 0 end) as Rotv_vl_avencer_181a360d 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_acima360d  else 0 end) as Rotv_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_361a720d   else 0 end) as Rotv_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_721a1080d   else 0 end) as Rotv_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304)then vl_avencer_1081a1440d   else 0 end) as Rotv_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_1441a1800d   else 0 end) as Rotv_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_1801a5400d   else 0 end) as Rotv_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer_acima_5400d   else 0 end) as Rotv_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_avencer            else 0 end) as Rotv_vl_avencer 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_venc_15d           else 0 end) as Rotv_vl_venc_15d 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_venc_30d           else 0 end) as Rotv_vl_venc_30d 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_venc_60d           else 0 end) as Rotv_vl_venc_60d 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_venc_90d           else 0 end) as Rotv_vl_venc_90d 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_sld_devedor_mensal else 0 end) as Rotv_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_sld_devedor_total  else 0 end) as Rotv_vl_sld_devedor_total
                    
       --MOD 401 1206 (Financiamento auto)******************************************************************************
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer30d_venc14d else 0 end) as Auto_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_30d        else 0 end) as Auto_vl_avencer_30d
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_31a60d     else 0 end) as Auto_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_61a90d     else 0 end) as Auto_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_91a180d    else 0 end) as Auto_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_181a360d   else 0 end) as Auto_vl_avencer_181a360d 
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_acima360d  else 0 end) as Auto_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_361a720d   else 0 end) as Auto_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_721a1080d   else 0 end) as Auto_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_1081a1440d   else 0 end) as Auto_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_1441a1800d   else 0 end) as Auto_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_1801a5400d   else 0 end) as Auto_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer_acima_5400d   else 0 end) as Auto_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (401,1206) then vl_avencer            else 0 end) as Auto_vl_avencer 
       ,sum(case when cod_modalidade in (401,1206) then vl_venc_15d           else 0 end) as Auto_vl_venc_15d 
       ,sum(case when cod_modalidade in (401,1206) then vl_venc_30d           else 0 end) as Auto_vl_venc_30d 
       ,sum(case when cod_modalidade in (401,1206) then vl_venc_60d           else 0 end) as Auto_vl_venc_60d 
       ,sum(case when cod_modalidade in (401,1206) then vl_venc_90d           else 0 end) as Auto_vl_venc_90d 
       ,sum(case when cod_modalidade in (401,1206) then vl_sld_devedor_mensal else 0 end) as Auto_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (401,1206) then vl_sld_devedor_total  else 0 end) as Auto_vl_sld_devedor_total
       
       --MOD 211 402 901 902 903 990 1202 (Financiamento Imobiliario)******************************************************************************
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer30d_venc14d else 0 end) as Imob_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_30d        else 0 end) as Imob_vl_avencer_30d
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_31a60d     else 0 end) as Imob_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_61a90d     else 0 end) as Imob_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_91a180d    else 0 end) as Imob_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_181a360d   else 0 end) as Imob_vl_avencer_181a360d 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_acima360d  else 0 end) as Imob_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_361a720d   else 0 end) as Imob_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_721a1080d   else 0 end) as Imob_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_1081a1440d   else 0 end) as Imob_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_1441a1800d   else 0 end) as Imob_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_1801a5400d   else 0 end) as Imob_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer_acima_5400d   else 0 end) as Imob_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_avencer            else 0 end) as Imob_vl_avencer 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_venc_15d           else 0 end) as Imob_vl_venc_15d 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_venc_30d           else 0 end) as Imob_vl_venc_30d 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_venc_60d           else 0 end) as Imob_vl_venc_60d 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_venc_90d           else 0 end) as Imob_vl_venc_90d 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_sld_devedor_mensal else 0 end) as Imob_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_sld_devedor_total  else 0 end) as Imob_vl_sld_devedor_total
       
       --MOD 211 401 402 801 802 901 902 903 990 1201 1202 1206 (Financiamento com Garantia)******************************************************************************
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer30d_venc14d else 0 end) as ComGar_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_30d        else 0 end) as ComGar_vl_avencer_30d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_31a60d     else 0 end) as ComGar_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_61a90d     else 0 end) as ComGar_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_91a180d    else 0 end) as ComGar_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_181a360d   else 0 end) as ComGar_vl_avencer_181a360d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_acima360d  else 0 end) as ComGar_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_361a720d   else 0 end) as ComGar_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_721a1080d   else 0 end) as ComGar_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_1081a1440d   else 0 end) as ComGar_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_1441a1800d   else 0 end) as ComGar_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_1801a5400d   else 0 end) as ComGar_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer_acima_5400d   else 0 end) as ComGar_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_avencer            else 0 end) as ComGar_vl_avencer 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_venc_15d           else 0 end) as ComGar_vl_venc_15d 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_venc_30d           else 0 end) as ComGar_vl_venc_30d 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_venc_60d           else 0 end) as ComGar_vl_venc_60d 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_venc_90d           else 0 end) as ComGar_vl_venc_90d 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_sld_devedor_mensal else 0 end) as ComGar_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_sld_devedor_total  else 0 end) as ComGar_vl_sld_devedor_total
       
       --MOD 202 (Emprestimo Consignado)******************************************************************************
       ,sum(case when cod_modalidade = 202 then vl_avencer30d_venc14d else 0 end) as Consig_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade = 202 then vl_avencer_30d        else 0 end) as Consig_vl_avencer_30d
       ,sum(case when cod_modalidade = 202 then vl_avencer_31a60d     else 0 end) as Consig_vl_avencer_31a60d 
       ,sum(case when cod_modalidade = 202 then vl_avencer_61a90d     else 0 end) as Consig_vl_avencer_61a90d 
       ,sum(case when cod_modalidade = 202 then vl_avencer_91a180d    else 0 end) as Consig_vl_avencer_91a180d 
       ,sum(case when cod_modalidade = 202 then vl_avencer_181a360d   else 0 end) as Consig_vl_avencer_181a360d 
       ,sum(case when cod_modalidade = 202 then vl_avencer_acima360d  else 0 end) as Consig_vl_avencer_acima360d
       ,sum(case when cod_modalidade = 202 then vl_avencer_361a720d   else 0 end) as Consig_vl_avencer_361a720d
       ,sum(case when cod_modalidade = 202 then vl_avencer_721a1080d   else 0 end) as Consig_vl_avencer_721a1080d
       ,sum(case when cod_modalidade = 202 then vl_avencer_1081a1440d   else 0 end) as Consig_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade = 202 then vl_avencer_1441a1800d   else 0 end) as Consig_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade = 202 then vl_avencer_1801a5400d   else 0 end) as Consig_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade = 202 then vl_avencer_acima_5400d   else 0 end) as Consig_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade = 202 then vl_avencer            else 0 end) as Consig_vl_avencer 
       ,sum(case when cod_modalidade = 202 then vl_venc_15d           else 0 end) as Consig_vl_venc_15d 
       ,sum(case when cod_modalidade = 202 then vl_venc_30d           else 0 end) as Consig_vl_venc_30d 
       ,sum(case when cod_modalidade = 202 then vl_venc_60d           else 0 end) as Consig_vl_venc_60d 
       ,sum(case when cod_modalidade = 202 then vl_venc_90d           else 0 end) as Consig_vl_venc_90d 
       ,sum(case when cod_modalidade = 202 then vl_sld_devedor_mensal else 0 end) as Consig_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade = 202 then vl_sld_devedor_total  else 0 end) as Consig_vl_sld_devedor_total
       
       --MOD 203 (Emprestimo Pessoal)******************************************************************************
       ,sum(case when cod_modalidade = 203 then vl_avencer30d_venc14d else 0 end) as EP_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade = 203 then vl_avencer_30d        else 0 end) as EP_vl_avencer_30d
       ,sum(case when cod_modalidade = 203 then vl_avencer_31a60d     else 0 end) as EP_vl_avencer_31a60d 
       ,sum(case when cod_modalidade = 203 then vl_avencer_61a90d     else 0 end) as EP_vl_avencer_61a90d 
       ,sum(case when cod_modalidade = 203 then vl_avencer_91a180d    else 0 end) as EP_vl_avencer_91a180d 
       ,sum(case when cod_modalidade = 203 then vl_avencer_181a360d   else 0 end) as EP_vl_avencer_181a360d 
       ,sum(case when cod_modalidade = 203 then vl_avencer_acima360d  else 0 end) as EP_vl_avencer_acima360d
       ,sum(case when cod_modalidade = 203 then vl_avencer_361a720d   else 0 end) as EP_vl_avencer_361a720d
       ,sum(case when cod_modalidade = 203 then vl_avencer_721a1080d   else 0 end) as EP_vl_avencer_721a1080d
       ,sum(case when cod_modalidade = 203 then vl_avencer_1081a1440d   else 0 end) as EP_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade = 203 then vl_avencer_1441a1800d   else 0 end) as EP_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade = 203 then vl_avencer_1801a5400d   else 0 end) as EP_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade = 203 then vl_avencer_acima_5400d   else 0 end) as EP_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade = 203 then vl_avencer            else 0 end) as EP_vl_avencer 
       ,sum(case when cod_modalidade = 203 then vl_venc_15d           else 0 end) as EP_vl_venc_15d 
       ,sum(case when cod_modalidade = 203 then vl_venc_30d           else 0 end) as EP_vl_venc_30d 
       ,sum(case when cod_modalidade = 203 then vl_venc_60d           else 0 end) as EP_vl_venc_60d 
       ,sum(case when cod_modalidade = 203 then vl_venc_90d           else 0 end) as EP_vl_venc_90d 
       ,sum(case when cod_modalidade = 203 then vl_sld_devedor_mensal else 0 end) as EP_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade = 203 then vl_sld_devedor_total  else 0 end) as EP_vl_sld_devedor_total
       
       --MOD 299 (Outros Emprestimos sem Garantias(refin,renegociação,composição,entre outros)))******************************************************************************
       ,sum(case when cod_modalidade = 299 then vl_avencer30d_venc14d else 0 end) as SemGarOut_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade = 299 then vl_avencer_30d        else 0 end) as SemGarOut_vl_avencer_30d
       ,sum(case when cod_modalidade = 299 then vl_avencer_31a60d     else 0 end) as SemGarOut_vl_avencer_31a60d 
       ,sum(case when cod_modalidade = 299 then vl_avencer_61a90d     else 0 end) as SemGarOut_vl_avencer_61a90d 
       ,sum(case when cod_modalidade = 299 then vl_avencer_91a180d    else 0 end) as SemGarOut_vl_avencer_91a180d 
       ,sum(case when cod_modalidade = 299 then vl_avencer_181a360d   else 0 end) as SemGarOut_vl_avencer_181a360d 
       ,sum(case when cod_modalidade = 299 then vl_avencer_acima360d  else 0 end) as SemGarOut_vl_avencer_acima360d
       ,sum(case when cod_modalidade = 299 then vl_avencer_361a720d   else 0 end) as SemGarOut_vl_avencer_361a720d
       ,sum(case when cod_modalidade = 299 then vl_avencer_721a1080d   else 0 end) as SemGarOut_vl_avencer_721a1080d
       ,sum(case when cod_modalidade = 299 then vl_avencer_1081a1440d   else 0 end) as SemGarOut_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade = 299 then vl_avencer_1441a1800d   else 0 end) as SemGarOut_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade = 299 then vl_avencer_1801a5400d   else 0 end) as SemGarOut_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade = 299 then vl_avencer_acima_5400d   else 0 end) as SemGarOut_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade = 299 then vl_avencer            else 0 end) as SemGarOut_vl_avencer 
       ,sum(case when cod_modalidade = 299 then vl_venc_15d           else 0 end) as SemGarOut_vl_venc_15d 
       ,sum(case when cod_modalidade = 299 then vl_venc_30d           else 0 end) as SemGarOut_vl_venc_30d 
       ,sum(case when cod_modalidade = 299 then vl_venc_60d           else 0 end) as SemGarOut_vl_venc_60d 
       ,sum(case when cod_modalidade = 299 then vl_venc_90d           else 0 end) as SemGarOut_vl_venc_90d 
       ,sum(case when cod_modalidade = 299 then vl_sld_devedor_mensal else 0 end) as SemGarOut_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade = 299 then vl_sld_devedor_total  else 0 end) as SemGarOut_vl_sld_devedor_total
       
       --MOD 202 203 299 (Emprestimo sem Garantia)******************************************************************************
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer30d_venc14d else 0 end) as SemGar_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_30d        else 0 end) as SemGar_vl_avencer_30d
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_31a60d     else 0 end) as SemGar_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_61a90d     else 0 end) as SemGar_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_91a180d    else 0 end) as SemGar_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_181a360d   else 0 end) as SemGar_vl_avencer_181a360d 
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_acima360d  else 0 end) as SemGar_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_361a720d   else 0 end) as SemGar_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_721a1080d   else 0 end) as SemGar_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_1081a1440d   else 0 end) as SemGar_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_1441a1800d   else 0 end) as SemGar_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_1801a5400d   else 0 end) as SemGar_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer_acima_5400d   else 0 end) as SemGar_vl_avencer_acima_5400d    
   
       ,sum(case when cod_modalidade in (202,203,299) then vl_avencer            else 0 end) as SemGar_vl_avencer 
       ,sum(case when cod_modalidade in (202,203,299) then vl_venc_15d           else 0 end) as SemGar_vl_venc_15d 
       ,sum(case when cod_modalidade in (202,203,299) then vl_venc_30d           else 0 end) as SemGar_vl_venc_30d 
       ,sum(case when cod_modalidade in (202,203,299) then vl_venc_60d           else 0 end) as SemGar_vl_venc_60d 
       ,sum(case when cod_modalidade in (202,203,299) then vl_venc_90d           else 0 end) as SemGar_vl_venc_90d 
       ,sum(case when cod_modalidade in (202,203,299) then vl_sld_devedor_mensal else 0 end) as SemGar_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (202,203,299) then vl_sld_devedor_total  else 0 end) as SemGar_vl_sld_devedor_total
       
       --MOD 202 203 207 211 250 290 299 401 402 404 405 440 450 490 499 501 502 503 504 590 599 601 702 801 802 803 890 901 902 903 990 1001 1101 1201 1202 1205 1206 1301 1302 1303 1350 1399 1401 (Emprestimo + Financiamentos)
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer30d_venc14d else 0 end) as EmpFin_vl_avencer30d_venc14d       
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_30d        else 0 end) as EmpFin_vl_avencer_30d
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_31a60d     else 0 end) as EmpFin_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_61a90d     else 0 end) as EmpFin_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_91a180d    else 0 end) as EmpFin_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_181a360d   else 0 end) as EmpFin_vl_avencer_181a360d
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_acima360d  else 0 end) as EmpFin_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_361a720d   else 0 end) as EmpFin_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_721a1080d   else 0 end) as EmpFin_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_1081a1440d   else 0 end) as EmpFin_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_1441a1800d   else 0 end) as EmpFin_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_1801a5400d   else 0 end) as EmpFin_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer_acima_5400d   else 0 end) as EmpFin_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_avencer            else 0 end) as EmpFin_vl_avencer 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_venc_15d           else 0 end) as EmpFin_vl_venc_15d 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_venc_30d           else 0 end) as EmpFin_vl_venc_30d 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_venc_60d           else 0 end) as EmpFin_vl_venc_60d 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_venc_90d           else 0 end) as EmpFin_vl_venc_90d 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_sld_devedor_mensal else 0 end) as EmpFin_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_sld_devedor_total  else 0 end) as EmpFin_vl_sld_devedor_total
       
       --MOD 202 203 211 299 401 402 403 499 1201 1206 1301 1401 (Emprestimo + Financiamentos (Principais))******************************************************************************
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer30d_venc14d else 0 end) as EmpFinPri_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_30d        else 0 end) as EmpFinPri_vl_avencer_30d
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_31a60d     else 0 end) as EmpFinPri_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_61a90d     else 0 end) as EmpFinPri_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_91a180d    else 0 end) as EmpFinPri_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_181a360d   else 0 end) as EmpFinPri_vl_avencer_181a360d 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_acima360d  else 0 end) as EmpFinPri_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_361a720d   else 0 end) as EmpFinPri_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_721a1080d   else 0 end) as EmpFinPri_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_1081a1440d   else 0 end) as EmpFinPri_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_1441a1800d   else 0 end) as EmpFinPri_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_1801a5400d   else 0 end) as EmpFinPri_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer_acima_5400d   else 0 end) as EmpFinPri_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_avencer            else 0 end) as EmpFinPri_vl_avencer 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_venc_15d           else 0 end) as EmpFinPri_vl_venc_15d 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_venc_30d           else 0 end) as EmpFinPri_vl_venc_30d 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_venc_60d           else 0 end) as EmpFinPri_vl_venc_60d 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_venc_90d           else 0 end) as EmpFinPri_vl_venc_90d 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_sld_devedor_mensal else 0 end) as EmpFinPri_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_sld_devedor_total  else 0 end) as EmpFinPri_vl_sld_devedor_total
       
       --MOD 206 215 216 217 (Capital de Giro)******************************************************************************
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer30d_venc14d else 0 end) as CGiro_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_30d        else 0 end) as CGiro_vl_avencer_30d
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_31a60d     else 0 end) as CGiro_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_61a90d     else 0 end) as CGiro_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_91a180d    else 0 end) as CGiro_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_181a360d   else 0 end) as CGiro_vl_avencer_181a360d 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_acima360d  else 0 end) as CGiro_vl_avencer_acima360d --MODIICAR
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_361a720d   else 0 end) as CGiro_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_721a1080d   else 0 end) as CGiro_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_1081a1440d   else 0 end) as CGiro_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_1441a1800d   else 0 end) as CGiro_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_1801a5400d   else 0 end) as CGiro_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer_acima_5400d   else 0 end) as CGiro_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_avencer            else 0 end) as CGiro_vl_avencer 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_venc_15d           else 0 end) as CGiro_vl_venc_15d 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_venc_30d           else 0 end) as CGiro_vl_venc_30d 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_venc_60d           else 0 end) as CGiro_vl_venc_60d 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_venc_90d           else 0 end) as CGiro_vl_venc_90d 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_sld_devedor_mensal else 0 end) as CGiro_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_sld_devedor_total  else 0 end) as CGiro_vl_sld_devedor_total
       
       --MOD 301 (Desconto de Duplicata)******************************************************************************
       ,sum(case when cod_modalidade = 301 then vl_avencer30d_venc14d else 0 end) as DesDup_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade = 301 then vl_avencer_30d        else 0 end) as DesDup_vl_avencer_30d
       ,sum(case when cod_modalidade = 301 then vl_avencer_31a60d     else 0 end) as DesDup_vl_avencer_31a60d 
       ,sum(case when cod_modalidade = 301 then vl_avencer_61a90d     else 0 end) as DesDup_vl_avencer_61a90d 
       ,sum(case when cod_modalidade = 301 then vl_avencer_91a180d    else 0 end) as DesDup_vl_avencer_91a180d 
       ,sum(case when cod_modalidade = 301 then vl_avencer_181a360d   else 0 end) as DesDup_vl_avencer_181a360d 
       ,sum(case when cod_modalidade = 301 then vl_avencer_acima360d  else 0 end) as DesDup_vl_avencer_acima360d
       ,sum(case when cod_modalidade = 301 then vl_avencer_361a720d   else 0 end) as DesDup_vl_avencer_361a720d
       ,sum(case when cod_modalidade = 301 then vl_avencer_721a1080d   else 0 end) as DesDup_vl_avencer_721a1080d
       ,sum(case when cod_modalidade = 301 then vl_avencer_1081a1440d   else 0 end) as DesDup_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade = 301 then vl_avencer_1441a1800d   else 0 end) as DesDup_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade = 301 then vl_avencer_1801a5400d   else 0 end) as DesDup_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade = 301 then vl_avencer_acima_5400d   else 0 end) as DesDup_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade = 301 then vl_avencer            else 0 end) as DesDup_vl_avencer 
       ,sum(case when cod_modalidade = 301 then vl_venc_15d           else 0 end) as DesDup_vl_venc_15d 
       ,sum(case when cod_modalidade = 301 then vl_venc_30d           else 0 end) as DesDup_vl_venc_30d 
       ,sum(case when cod_modalidade = 301 then vl_venc_60d           else 0 end) as DesDup_vl_venc_60d 
       ,sum(case when cod_modalidade = 301 then vl_venc_90d           else 0 end) as DesDup_vl_venc_90d 
       ,sum(case when cod_modalidade = 301 then vl_sld_devedor_mensal else 0 end) as DesDup_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade = 301 then vl_sld_devedor_total  else 0 end) as DesDup_vl_sld_devedor_total
       
       --MOD 302 (Desconto de Cheque)******************************************************************************
       ,sum(case when cod_modalidade = 302 then vl_avencer30d_venc14d else 0 end) as DesChq_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade = 302 then vl_avencer_30d        else 0 end) as DesChq_vl_avencer_30d
       ,sum(case when cod_modalidade = 302 then vl_avencer_31a60d     else 0 end) as DesChq_vl_avencer_31a60d 
       ,sum(case when cod_modalidade = 302 then vl_avencer_61a90d     else 0 end) as DesChq_vl_avencer_61a90d 
       ,sum(case when cod_modalidade = 302 then vl_avencer_91a180d    else 0 end) as DesChq_vl_avencer_91a180d 
       ,sum(case when cod_modalidade = 302 then vl_avencer_181a360d   else 0 end) as DesChq_vl_avencer_181a360d 
       ,sum(case when cod_modalidade = 302 then vl_avencer_acima360d  else 0 end) as DesChq_vl_avencer_acima360d
       ,sum(case when cod_modalidade = 302 then vl_avencer_361a720d   else 0 end) as DesChq_vl_avencer_361a720d
       ,sum(case when cod_modalidade = 302 then vl_avencer_721a1080d   else 0 end) as DesChq_vl_avencer_721a1080d
       ,sum(case when cod_modalidade = 302 then vl_avencer_1081a1440d   else 0 end) as DesChq_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade = 302 then vl_avencer_1441a1800d   else 0 end) as DesChq_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade = 302 then vl_avencer_1801a5400d   else 0 end) as DesChq_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade = 302 then vl_avencer_acima_5400d   else 0 end) as DesChq_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade = 302 then vl_avencer            else 0 end) as DesChq_vl_avencer 
       ,sum(case when cod_modalidade = 302 then vl_venc_15d           else 0 end) as DesChq_vl_venc_15d 
       ,sum(case when cod_modalidade = 302 then vl_venc_30d           else 0 end) as DesChq_vl_venc_30d 
       ,sum(case when cod_modalidade = 302 then vl_venc_60d           else 0 end) as DesChq_vl_venc_60d 
       ,sum(case when cod_modalidade = 302 then vl_venc_90d           else 0 end) as DesChq_vl_venc_90d 
       ,sum(case when cod_modalidade = 302 then vl_sld_devedor_mensal else 0 end) as DesChq_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade = 302 then vl_sld_devedor_total  else 0 end) as DesChq_vl_sld_devedor_total
       
       --MOD 301 302 303 398 399 (Desconto)******************************************************************************
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer30d_venc14d else 0 end) as Descont_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_30d        else 0 end) as Descont_vl_avencer_30d
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_31a60d     else 0 end) as Descont_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_61a90d     else 0 end) as Descont_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_91a180d    else 0 end) as Descont_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_181a360d   else 0 end) as Descont_vl_avencer_181a360d 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_acima360d  else 0 end) as Descont_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_361a720d   else 0 end) as Descont_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_721a1080d   else 0 end) as Descont_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_1081a1440d   else 0 end) as Descont_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_1441a1800d   else 0 end) as Descont_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_1801a5400d   else 0 end) as Descont_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer_acima_5400d   else 0 end) as Descont_vl_avencer_acima_5400d
   
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_avencer            else 0 end) as Descont_vl_avencer 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_venc_15d           else 0 end) as Descont_vl_venc_15d 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_venc_30d           else 0 end) as Descont_vl_venc_30d 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_venc_60d           else 0 end) as Descont_vl_venc_60d 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_venc_90d           else 0 end) as Descont_vl_venc_90d 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_sld_devedor_mensal else 0 end) as Descont_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_sld_devedor_total  else 0 end) as Descont_vl_sld_devedor_total
       
       --MOD 212 403 (Microcrédito)******************************************************************************
       ,sum(case when cod_modalidade in (212,403) then vl_avencer30d_venc14d else 0 end) as Micro_vl_avencer30d_venc14d
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_30d        else 0 end) as Micro_vl_avencer_30d
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_31a60d     else 0 end) as Micro_vl_avencer_31a60d 
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_61a90d     else 0 end) as Micro_vl_avencer_61a90d 
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_91a180d    else 0 end) as Micro_vl_avencer_91a180d 
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_181a360d   else 0 end) as Micro_vl_avencer_181a360d 
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_acima360d  else 0 end) as Micro_vl_avencer_acima360d
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_361a720d   else 0 end) as Micro_vl_avencer_361a720d
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_721a1080d   else 0 end) as Micro_vl_avencer_721a1080d
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_1081a1440d   else 0 end) as Micro_vl_avencer_1081a1440d
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_1441a1800d   else 0 end) as Micro_vl_avencer_1441a1800d
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_1801a5400d   else 0 end) as Micro_vl_avencer_1801a5400d
       ,sum(case when cod_modalidade in (212,403) then vl_avencer_acima_5400d   else 0 end) as Micro_vl_avencer_acima_5400d
       
       ,sum(case when cod_modalidade in (212,403) then vl_avencer            else 0 end) as Micro_vl_avencer 
       ,sum(case when cod_modalidade in (212,403) then vl_venc_15d           else 0 end) as Micro_vl_venc_15d 
       ,sum(case when cod_modalidade in (212,403) then vl_venc_30d           else 0 end) as Micro_vl_venc_30d 
       ,sum(case when cod_modalidade in (212,403) then vl_venc_60d           else 0 end) as Micro_vl_venc_60d 
       ,sum(case when cod_modalidade in (212,403) then vl_venc_90d           else 0 end) as Micro_vl_venc_90d 
       ,sum(case when cod_modalidade in (212,403) then vl_sld_devedor_mensal else 0 end) as Micro_vl_sld_devedor_mensal 
       ,sum(case when cod_modalidade in (212,403) then vl_sld_devedor_total  else 0 end) as Micro_vl_sld_devedor_total
       
        --VALOR LIMITE DISPONIVEL*********************************************************************************************************************************************
       ,sum(case when cod_modalidade in (1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1999) and vl_limite_disponivel is not null then vl_limite_disponivel 
                 when cod_modalidade in (101, 213, 214, 204, 218, 210, 406, 1304) then 0
                 else null end) as vl_limite_disponivel
                 
       --ROTATIVOS - Limite Total (vl_limite_disponivel + vl_sld_devedor_total rotativos)  ***********************************************************************************        
       ,COALESCE(sum(case when cod_modalidade in (1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1999) and vl_limite_disponivel is not null then vl_limite_disponivel 
                          when cod_modalidade in (101, 213, 214, 204, 218, 210, 406, 1304) then 0
                          else null end),0) 
       + COALESCE(sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_sld_devedor_total  else 0 end),0) as vl_limite
       
       --ROTATIVOS - Utilização (vl_limite - vl_limite_disponivel) / vl_limite **********************************************************************************************
       ,(COALESCE(sum(case when cod_modalidade in (1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1999) and vl_limite_disponivel is not null then vl_limite_disponivel 
                          when cod_modalidade in (101, 213, 214, 204, 218, 210, 406, 1304) then 0
                          else null end),0) 
       + COALESCE(sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_sld_devedor_total  else 0 end),0) 
       - sum(case when cod_modalidade in (1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1999) and vl_limite_disponivel is not null then vl_limite_disponivel 
                 when cod_modalidade in (101, 213, 214, 204, 218, 210, 406, 1304) then 0
                 else null end))
       /(COALESCE(sum(case when cod_modalidade in (1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1999) and vl_limite_disponivel is not null then vl_limite_disponivel 
                          when cod_modalidade in (101, 213, 214, 204, 218, 210, 406, 1304) then 0
                          else null end),0) 
       + COALESCE(sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_sld_devedor_total  else 0 end),0)) as Pc_Utiliz_limite 
       
       --ROTATIVOS - Saldo Rotativo / Saldo devedor Mensal ******************************************************************************************************************
       ,COALESCE(sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_sld_devedor_mensal else 0 end),0) / COALESCE(sum(vl_sld_devedor_mensal),0) as Pc_Sd_Rot_Por_Mensal  
       
       --CALCULOS BACEN*****************************************************************************************
       ,sum(Bacen_vl_carteira_ativa) as Bacen_vl_carteira_ativa
       ,sum(Bacen_vl_carteira_credito) as Bacen_vl_carteira_credito
       ,sum(Bacen_vl_relac_interfin) as Bacen_vl_relac_interfin
       ,sum(Bacen_vl_gar_prestadas) as Bacen_vl_gar_prestadas
       ,sum(Bacen_vl_coobigacao_assumida) as Bacen_vl_coobigacao_assumida
       ,sum(Bacen_vl_coobricacao) as Bacen_vl_coobricacao
       ,sum(Bacen_vl_resp_total) as Bacen_vl_resp_total
       ,sum(Bacen_vl_limite_credito) as Bacen_vl_limite_credito
       ,sum(Bacen_vl_creditos_a_liberar) as Bacen_vl_creditos_a_liberar
       ,sum(Bacen_risco_indireto) as Bacen_risco_indireto
       ,sum(Bacen_risco_total) as Bacen_risco_total
       ,max(qtdIF) as qtdIF
       ,max(meses_relac_if) as meses_relac_if
       
       --PERCENTUAIS***************************************************************************************** --MODIFICAR???
       ,sum(vl_avencer_30d)        / sum(Bacen_risco_total) as Pc_avencer_30d_risco_total
       ,sum(vl_avencer_31a60d)     / sum(Bacen_risco_total) as Pc_avencer_31a60d_risco_total
       ,sum(vl_avencer_61a90d)     / sum(Bacen_risco_total) as Pc_avencer_61a90d_risco_total
       ,sum(vl_avencer_91a180d)    / sum(Bacen_risco_total) as Pc_avencer_91a180d_risco_total
       ,sum(vl_avencer_181a360d)   / sum(Bacen_risco_total) as Pc_avencer_181a360d_risco_total
       ,sum(vl_avencer)            / sum(Bacen_risco_total) as Pc_avencer_risco_total
       ,sum(vl_venc_1d)            / sum(Bacen_risco_total) as Pc_venc_1d_risco_total
       ,sum(vl_prejuizo)           / sum(Bacen_risco_total) as Pc_prejuizo_risco_total
       ,sum(vl_avencer30d_venc14d) / sum(Bacen_risco_total) as Pc_avencer30d_venc14d_risco_total
       ,sum(vl_avencer_venc14d)    / sum(Bacen_risco_total) as Pc_avencer_venc14d_risco_total
       ,sum(vl_venc_15d)           / sum(Bacen_risco_total) as Pc_venc_15d_risco_total
       ,sum(vl_venc_30d)           / sum(Bacen_risco_total) as Pc_venc_30d_risco_total
       ,sum(vl_venc_60d)           / sum(Bacen_risco_total) as Pc_venc_60d_risco_total
       ,sum(vl_venc_90d)           / sum(Bacen_risco_total) as Pc_venc_90d_risco_total
       ,sum(vl_venc_120d)          / sum(Bacen_risco_total) as Pc_venc_120d_risco_total
       ,sum(vl_venc_180d)          / sum(Bacen_risco_total) as Pc_venc_180d_risco_total
       ,sum(vl_prejuizo_ult48m)    / sum(Bacen_risco_total) as Pc_prejuizo_ult48m_risco_total
              
       ,sum(vl_sld_devedor_mensal)/ sum(Bacen_risco_total) as Pc_devedor_mensal_risco_total
       ,sum(vl_sld_devedor_total)/ sum(Bacen_risco_total) as Pc_devedor_total_risco_total
       
       ,sum(case when cod_modalidade = 101 then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_Adp_devedor_mensal_risco_total
       ,sum(case when cod_modalidade = 101 then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_Adp_devedor_total_risco_total
       ,sum(case when cod_modalidade in (201,213,214) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_chqCcg_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (201,213,214) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_chqCcg_devedor_total_risco_total
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_AdpChqCcg_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (101,201,213,214) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_AdpChqCcg_devedor_total_risco_total
       ,sum(case when cod_modalidade in (204,218) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_CarRot_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (204,218) then vl_sld_devedor_total  else 0 end)  / sum(Bacen_risco_total) as Pc_CarRot_devedor_total_risco_total
       ,sum(case when cod_modalidade in (210,406) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_CarSaqLoj_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (210,406) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_CarSaqLoj_devedor_total_risco_total
       ,sum(case when cod_modalidade = 1304 then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_CarVisPar_devedor_mensal_risco_total
       ,sum(case when cod_modalidade = 1304 then vl_sld_devedor_total  else 0 end)  / sum(Bacen_risco_total) as Pc_CarVisPar_devedor_total_risco_total
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_Car_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (204,218,210,406,1304) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_Car_devedor_total_risco_total
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_Rotv_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (101,201,213,214,204,218,210,406,1304) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_Rotv_devedor_total_risco_total
       ,sum(case when cod_modalidade in (401,1206) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_Auto_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (401,1206) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_Auto_devedor_total_risco_total
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_Imob_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (211,402,901,902,903,990,1202) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_Imob_devedor_total_risco_total
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_ComGar_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (211,401,402,801,802,901,902,903,990,1201,1202,1206) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_ComGar_devedor_total_risco_total
       ,sum(case when cod_modalidade = 202 then vl_sld_devedor_mensal else 0 end) / sum(Bacen_risco_total) as Pc_Consig_devedor_mensal_risco_total
       ,sum(case when cod_modalidade = 202 then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_Consig_devedor_total_risco_total
       ,sum(case when cod_modalidade = 203 then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_EP_devedor_mensal_risco_total
       ,sum(case when cod_modalidade = 203 then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_EP_devedor_total_risco_total
       ,sum(case when cod_modalidade = 299 then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_SemGarOut_devedor_mensal_risco_total
       ,sum(case when cod_modalidade = 299 then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_SemGarOut_devedor_total_risco_total
       ,sum(case when cod_modalidade in (202,203,299) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_SemGar_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (202,203,299) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_SemGar_devedor_total_risco_total
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_EmpFin_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (202,203,207,211,250,290,299,401,402,404,405,440,450,490,499,501,502,503,504,590,599,601,702,801,802,803,890,901,902,903,990,1001,1101,1201,1202,1205,1206,1301,1302,1303,1350,1399,1401) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_EmpFin_devedor_total_risco_total
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_EmpFinPri_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (202,203,211,299,401,402,403,499,1201,1206,1301,1401) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_EmpFinPri_devedor_total_risco_total
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_sld_devedor_mensal else 0 end)/ sum(Bacen_risco_total) as Pc_CGiro_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (206,215,216,217) then vl_sld_devedor_total  else 0 end)  / sum(Bacen_risco_total) as Pc_CGiro_devedor_total_risco_total
       ,sum(case when cod_modalidade = 301 then vl_sld_devedor_mensal else 0 end) / sum(Bacen_risco_total) as Pc_DesDup_devedor_mensal_risco_total
       ,sum(case when cod_modalidade = 301 then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_DesDup_devedor_total_risco_total
       ,sum(case when cod_modalidade = 302 then vl_sld_devedor_mensal else 0 end) / sum(Bacen_risco_total) as Pc_DesChq_devedor_mensal_risco_total
       ,sum(case when cod_modalidade = 302 then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_DesChq_devedor_total_risco_total
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_sld_devedor_mensal else 0 end) / sum(Bacen_risco_total) as Pc_Descont_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (301,302,303,398,399) then vl_sld_devedor_total  else 0 end)/ sum(Bacen_risco_total) as Pc_Descont_devedor_total_risco_total
       ,sum(case when cod_modalidade in (212,403) then vl_sld_devedor_mensal else 0 end)  / sum(Bacen_risco_total) as Pc_Micro_devedor_mensal_risco_total
       ,sum(case when cod_modalidade in (212,403) then vl_sld_devedor_total  else 0 end) / sum(Bacen_risco_total) as Pc_Micro_devedor_total_risco_total

       ,sum(Bacen_vl_carteira_ativa) / sum(Bacen_risco_total) as Pc_carteira_ativa_risco_total
       ,sum(Bacen_vl_carteira_credito) / sum(Bacen_risco_total) as Pc_carteira_credito_risco_total
       ,sum(Bacen_vl_relac_interfin) / sum(Bacen_risco_total) as Pc_relac_interfin_risco_total
       ,sum(Bacen_vl_gar_prestadas) / sum(Bacen_risco_total) as Pc_gar_prestadas_risco_total
       ,sum(Bacen_vl_coobigacao_assumida) / sum(Bacen_risco_total) as Pc_coobigacao_assumida_risco_total
       ,sum(Bacen_vl_coobricacao) / sum(Bacen_risco_total) as Pc_coobricacao_risco_total
       ,sum(Bacen_vl_resp_total) / sum(Bacen_risco_total) as Pc_resp_total_risco_total
       ,sum(Bacen_vl_limite_credito) / sum(Bacen_risco_total) as Pc_limite_credito_risco_total
       ,sum(Bacen_vl_creditos_a_liberar) / sum(Bacen_risco_total) as Pc_creditos_a_liberar_risco_total
       ,sum(Bacen_risco_indireto) / sum(Bacen_risco_total) as Pc_risco_indireto_risco_total
  from ''' + TABELA_SCR_AUX + '''
 group by 1, 2, 3, 4, 5
'''

spark.sql(query_book_scr)

num_affected_rows,num_inserted_rows


In [0]:
caminho_tabela_descricao = 'default.book_scr_202109'
query_describe = '''describe default.book_scr_202109'''

tabela_descricao = spark.sql(query_describe).toPandas()

for idx in range(0, len(tabela_descricao)-3):
  col_nome = tabela_descricao.loc[idx, 'col_name']
  col_tipo = tabela_descricao.loc[idx, 'data_type']
  col_descricao = tabela_descricao.loc[idx, 'comment']
  
  query_alter = 'ALTER TABLE '+TABELA_SCR_FINAL+' CHANGE '+col_nome+' '+col_nome+' '+col_tipo+' COMMENT '+"'"+col_descricao+"'"
  spark.sql(query_alter)
  #print(query_alter)